<a href="https://colab.research.google.com/github/hbarze/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Deliverable 2: Determining the Bias of Vine Reviews
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done


In [5]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-04 22:28:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2022-05-04 22:28:08 (1.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [7]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [8]:
df.count()

1785997

In [12]:
#Loading in sql function to use columns 
from pyspark.sql.functions import col

# Filtering for columns where total_votes are greater than or equal to 20
total_votes_20_df = df.filter(col("total_votes") >= 20)
total_votes_20_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [10]:
total_votes_20_df.count()

65379

In [11]:
#Filtering the data to retrieve all rows where the number of helpful_votes divided by total_votes is equal or greater than 50%
total_votes_50_df = total_votes_20_df.filter(col("helpful_votes")/col("total_votes") >= .50)
total_votes_50_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [13]:
total_votes_50_df.count()

40565

In [14]:
#Filtering for data where a review was written as part of the Vine program (paid), where vine == 'Y'
paid_vine_df = total_votes_50_df.filter(col("vine") == 'Y')
paid_vine_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48771843|R3KKUSGFZWSUIY|B00YXO5UXG|     555176890|Turtle Beach - Ea...|     Video Games|          5|           56|         63|   Y|                N|Quality Chat Headset|Not every situati...| 2015-08-04|
|         US|   53080186|R10FO5UKKVZBK2|B00XO041RQ|     238654494|PDP AG7 True Wire...|     Video Games|          3|    

In [15]:
paid_vine_df.count()

94

In [16]:
#Filter the data again for all rows where the review was not part of the Vine program (unpaid), where vine == 'N'
unpaid_vine_df = total_votes_50_df.filter(col("vine") == 'N')
unpaid_vine_df.show(5)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [17]:
unpaid_vine_df.count()

40471

In [20]:
# Filtering the data to find the percentage of paid 5-star reviews
total_fivestar_reviews = paid_vine_df.filter(paid_vine_df.star_rating == 5).count()
total_fivestar_reviews

48

In [22]:
#Filtering the data to find the percentage of 5-star reviews for paid reviews
paid_fivestar_percent = (total_fivestar_reviews / paid_vine_df.count()) * 100
paid_fivestar_percent

51.06382978723404

In [23]:
#Filtering the data to find the percentage of 5-star reviews for unpaid reviews
unpaid_fivestar_percent = (total_fivestar_reviews / unpaid_vine_df.count()) * 100
unpaid_fivestar_percent

0.11860344444169899